In [1]:
import networkx as nx
import numpy as np

In [2]:
import mercs

In [3]:
from mercs.utils import code_to_query

from mercs.graph.q_diagram import build_diagram

# Preliminaries

Getting a very basic mercs model on my hands to test with.

In [4]:
m_codes = [
    [ 0, 0,-1,-1 ,1],
    [1,-1, 0, 0, 1],
    [-1,-1, 1,-1, 0],
    [-1,-1, 0, 1, 0]  
]

m_codes = np.array(m_codes)
m_codes

array([[ 0,  0, -1, -1,  1],
       [ 1, -1,  0,  0,  1],
       [-1, -1,  1, -1,  0],
       [-1, -1,  0,  1,  0]])

In [5]:
class model():
    def __init__(self, m_code):
        self.desc_ids = set(np.where(m_code==0)[0])
        self.targ_ids = set(np.where(m_code==1)[0])
        return

In [6]:
m_list = [model(m_code) for m_code in m_codes]
m_list[0].desc_ids

{0, 1}

In [7]:
from functools import reduce

# Algorithm

## Single-Layer Model

In [8]:
m_sel = [np.array([0,1])]
q_code = np.array([1, 0, 0, 0, 1])


#g.nodes()

In [9]:
%%timeit
g = build_diagram(m_list, m_sel, q_code, g=None, prune=True)

75.7 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
g = build_diagram(m_list, m_sel, q_code, g=None, prune=True)

In [11]:
g.nodes

NodeView((('I', 0), ('D', 0), ('D', 3), ('M', 1), ('D', 1), ('M', 0), ('D', 2), ('D', 4)))

In [12]:
for n, idx in :
    print(n)
    print(idx)

SyntaxError: invalid syntax (<ipython-input-12-033fba5e254e>, line 1)

In [ ]:
g.predecessors(('D', 4))

In [ ]:
g.remove_nodes_from

In [ ]:
list(nx.topological_sort(g))

In [ ]:
nx.ancestors(g, ('M', 1))

Change inputs in dask is still absolutely crucial. But this network way of viewing ML as a whole could be a very, very cool idea. Especially if you want the problem to be treated recursively. Please persist in your efforts. In the end, the idea would be to do ML on an unprecendented scale, decentralized yet coordinated.

# Inference

In [ ]:
g.in_degree()

In [ ]:
g.add_nodes_from([('elia', dict(last='van woplutte'))])

In [ ]:
g.nodes(data=True)